In [1]:
import scipy as sp
import numpy as np
import os 
import pickle

import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

from architectures.cycleGAN import *

In [2]:
import tensorflow as tf
tf.__version__

'1.3.0'

In [3]:
def insert(X_inner, X_outer):
    X=X_outer
    
    for i in range(26,78):
        for j in range(32,96):
            X[i,j]=X_inner[i-26,j-36]
    
    return X

In [4]:
# some constants

LEARNING_RATE_D = 0.0001
LEARNING_RATE_G = 0.0001
BETA1 = 0.5
BATCH_SIZE = 128
EPOCHS = 5
SAVE_SAMPLE_PERIOD = 100
task='TRAIN'
#task='TEST'

PATH = 'cycleGAN_test'

In [5]:
trunc_normal= tf.truncated_normal_initializer(stddev=0.02)
normal = tf.random_normal_initializer(stddev=0.02)

uniform = tf.random_uniform_initializer()
glorot_uniform = tf.glorot_uniform_initializer()
glorot_normal = tf.glorot_normal_initializer()

global d_sizes, g_sizes

d_sizes_A = {
         'conv_layers': [(2, 6, 2, False, 1, lrelu, trunc_normal),
                         (64, 6, 2, True, 1, lrelu, trunc_normal)],
         'dense_layers': [(1024, True, 1, lrelu ,normal)],
         'readout_layer_w_init':normal,
}

d_sizes_B = {
         'conv_layers': [(2, 6, 2, False, 1, lrelu, trunc_normal),
                         (64, 6, 2, True, 1, lrelu, trunc_normal)],
         'dense_layers': [(1024, True, 1, lrelu ,normal)],
         'readout_layer_w_init':normal,
}

g_sizes_A={
         'conv_layer_0':[(4, 8, 2, False, 1, tf.nn.relu, normal)],
         'conv_layer_1':[(8, 8, 2, False, 1, tf.nn.relu, normal)],
         'convblock_layer_0':[(16, 4, 2, True, 1, tf.nn.relu, normal),
                                (16, 4, 1, False, 1, tf.nn.relu, normal),
                                (16, 4, 2, True, 1, tf.nn.relu, normal)],
         'convblock_shortcut_layer_0':[(16, 1, 4, False, 1, normal)],
         'deconv_layer_0':[(8, 8, 2, False, 1, tf.nn.relu, normal)],
         'deconv_layer_1':[(1, 8, 2, False, 1, tf.nn.relu, normal)],
         
        
         
}

g_sizes_B={
         'conv_layer_0':[(4, 8, 2, False, 1, tf.nn.relu, normal)],
         'conv_layer_1':[(8, 8, 2, False, 1, tf.nn.relu, normal)],
         'convblock_layer_0':[(16, 4, 2, True, 1, tf.nn.relu, normal),
                                (16, 4, 1, False, 1, tf.nn.relu, normal),
                                (16, 4, 2, True, 1, tf.nn.relu, normal)],
         'convblock_shortcut_layer_0':[(16, 1, 4, False, 1, normal)],
         'deconv_layer_0':[(8, 8, 2, False, 1, tf.nn.relu, normal)],
         'deconv_layer_1':[(1, 8, 2, False, 1, tf.nn.relu, normal)],
        
         
}



In [6]:
def HCAL():

    with open('samples/reco/sample0.pickle', 'rb') as f:
        reco=pickle.load(f)
    #with open('samples/reco/sample1.pickle', 'rb') as f:
    #    reco1=pickle.load(f)
        
    #reco=np.concatenate((reco,reco1),axis=0)
    
    pos=[]
    for i in range(len(reco)):
        if np.array_equal(reco[i],np.zeros_like(reco[i])):
            pos.append(i)
    reco = np.delete(reco, pos, axis=0)
    reco[reco!=0]/=np.max(reco)
    X_train_B=reco
    
    with open('samples/true/sample0.pickle', 'rb') as f:
        true=pickle.load(f)
    true = true[:,1:true.shape[1]-1,1:true.shape[2]-1]
    #with open('samples/true/sample1.pickle', 'rb') as f:
    #    true1=pickle.load(f)
        
    #true=np.concatenate((true,true1),axis=0)

    true = np.delete(true, pos, axis=0)
    true[true!=0]/=np.max(true)
    
    X_train_A = true
    
    m, n_H, n_W, n_C = X_train_A.shape
    
    #test_size = 5000
    #train_size = m - test_size
    
    out = np.zeros_like(X_train_A)
    
    for i in range(len(out)):
        out[i] = insert(X_train_B[i],out[i])
    
    X_train_B = out
    
    tf.reset_default_graph()

    gan = cycleGAN(n_H, n_W, n_C, d_sizes_A, d_sizes_B, g_sizes_A, g_sizes_B,
                   lr_g=LEARNING_RATE_G, lr_d=LEARNING_RATE_D,beta1=BETA1,
                  batch_size=BATCH_SIZE, epochs=EPOCHS,
                  save_sample=SAVE_SAMPLE_PERIOD, path=PATH)
    
    vars_to_train= tf.trainable_variables()
    
    if task == 'TRAIN':
        init_op = tf.global_variables_initializer()
        
    if task == 'TEST':
        vars_all = tf.global_variables()
        vars_to_init = list(set(vars_all)-set(vars_to_train))
        init_op = tf.variables_initializer(vars_to_init)
        
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()    
        
    with tf.Session() as sess:
        
        sess.run(init_op)

        if task=='TRAIN':
            print('\n Training...')
            
            if os.path.exists(PATH+'/'+PATH+'.ckpt.index'):
                saver.restore(sess,PATH+'/'+PATH+'.ckpt')
                print('Model restored.')
            
            gan.set_session(sess)
            gan.fit(X_train_A, X_train_B)
            
            save_path = saver.save(sess, PATH+'/'+PATH+'.ckpt')
            print("Model saved in path: %s" % save_path)
        
        if task=='TEST':
            print('\n Evaluate model on test set...')
            saver.restore(sess,PATH+'/'+PATH+'.ckpt')
            print('Model restored.')
            
            gan.set_session(sess) 
            
        done = True
        while not done:
            
            
            Z_in = np.random.uniform(-1,1, size=(1, g_sizes['z']))
            
            im = gan.get_sample(Z_in)
            
            plt.imshow(im.reshape(28,28), cmap='gray')
            plt.show()
            
            
            ans = input("Generate another?")
            if ans and ans[0] in ('n' or 'N'):
                done = True

In [7]:
if __name__=='__main__':

    if task == 'TRAIN':
        if not os.path.exists(PATH):
            os.mkdir(PATH)
    
        elif os.path.exists(PATH):
            if os.path.exists(PATH+'/checkpoint'):
                ans = input('A previous checkpoint already exists, choose the action to perform \n \n 1) Overwrite the current model saved at '+PATH+'/checkpoint \n 2) Start training a new model \n 3) Restore and continue training the previous model \n ')
                
                if ans == '1':
                    print('Overwriting existing model in '+PATH)
                    for file in os.listdir(PATH):
                        file_path = os.path.join(PATH, file)
                        try:
                            if os.path.isfile(file_path):
                                os.unlink(file_path)
                            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
                        except Exception as e:
                            print(e)
                            
                elif ans == '2':
                    PATH = input('Specify the name of the model, a new directory will be created.\n')
                    os.mkdir(PATH)    
        
        HCAL()
   
    elif task == 'TEST': 
        if not os.path.exists(PATH+'/checkpoint'):
            print('No checkpoint to test')
        else:
            HCAL()

Convolutional Network architecture detected
Convolutional Network architecture detected
Discriminator_A
Convolution
Input for convolution shape  (?, 104, 128, 1)
(53248, 1024)
Logits shape (?, 1)
Generator_A_to_B
Input for deconvolution shape (?, 104, 128, 1)
Deconvoluted output shape (?, 104, 128, 1)
Discriminator_B
Convolution
Input for convolution shape  (?, 104, 128, 1)
(53248, 1024)
Logits shape (?, 1)
Generator_B_to_A
Input for deconvolution shape (?, 104, 128, 1)
Deconvoluted output shape (?, 104, 128, 1)
Discriminator_B
Convolution
Input for convolution shape  (?, 104, 128, 1)
(53248, 1024)
Logits shape (?, 1)
Generator_B_to_A
Input for deconvolution shape (?, 104, 128, 1)
Deconvoluted output shape (?, 104, 128, 1)
Discriminator_A
Convolution
Input for convolution shape  (?, 104, 128, 1)
(53248, 1024)
Logits shape (?, 1)
Generator_A_to_B
Input for deconvolution shape (?, 104, 128, 1)
Deconvoluted output shape (?, 104, 128, 1)
Generator_A_to_B
Input for deconvolution shape (?, 1

NameError: name 'X_train' is not defined

In [ ]:
    with open('samples/reco/sample0.pickle', 'rb') as f:
        reco=pickle.load(f)
    #with open('samples/reco/sample1.pickle', 'rb') as f:
    #    reco1=pickle.load(f)
        
    #reco=np.concatenate((reco,reco1),axis=0)
    
    pos=[]
    for i in range(len(reco)):
        if np.array_equal(reco[i],np.zeros_like(reco[i])):
            pos.append(i)
    reco = np.delete(reco, pos, axis=0)
    reco[reco!=0]/=np.max(reco)
    X_train_B=reco
    
    with open('samples/true/sample0.pickle', 'rb') as f:
        true=pickle.load(f)
    true = true[:,1:true.shape[1]-1,1:true.shape[2]-1]
    #with open('samples/true/sample1.pickle', 'rb') as f:
    #    true1=pickle.load(f)
        
    #true=np.concatenate((true,true1),axis=0)

    true = np.delete(true, pos, axis=0)
    true[true!=0]/=np.max(true)
    
    X_train_A = true
    
    m, n_H, n_W, n_C = X_train_A.shape
    
    #test_size = 5000
    #train_size = m - test_size
    
    out = np.zeros_like(X_train_A)
    
    for i in range(len(out)):
        out[i] = insert(X_train_B[i],out[i])
    
    X_train_B = out
        

In [ ]:
X_train_A.shape

In [ ]:
X_train_B.shape

In [ ]:
out = np.zeros_like(X_train_A)

In [ ]:
def insert(X_inner, X_outer):
    X=X_outer
    
    for i in range(26,78):
        for j in range(32,96):
            X[i,j]=X_inner[i-26,j-36]
    
    return X

In [ ]:
for i in range(len(out)):
    out[i] = insert(X_train_B[i],out[i])

In [ ]:
plt.imshow(out[1000].reshape(104,128),cmap='gray')

In [ ]:
plt.imshow(X_train_A[1000].reshape(104,128),cmap='gray')

In [ ]:
g_sizes_A={
         'conv_layer_0':[()],
         'conv_layer_1':[()],
    
         'convblock_layer_0':[(64, 4, 2, True, 1, tf.nn.relu, normal),
                                (8, 4, 1, False, 1, tf.nn.relu, normal),
                                (1, 4, 2, False, 1, tf.sigmoid, normal)],
         'convblock_shortcut_layer_0':[(1, 1, 4, False, 1, normal)],
    
         'convblock_layer_1':[(64, 4, 2, True, 1, tf.nn.relu, normal),
                                (8, 4, 1, False, 1, tf.nn.relu, normal),
                                (1, 4, 2, False, 1, tf.sigmoid, normal)],
         'convblock_shortcut_layer_1':[(1, 1, 4, False, 1, normal)],
    
         'deconv_layer_0':[()],
         'deconv_layer_1':[()],
        
         
}

In [ ]:
                g_steps=0
                for key in g_sizes_A:
                    g_steps+=1

                g_convs = 0
                g_deconvs = 0

                g_conv_blocks = 0
                #g_deconv_blocks = 0

                for key in g_sizes_A:

                    if 'conv' in key:
                        if not 'deconv' in key:
                            if not 'shortcut' in key:
                                if not 'block' in key:
                                    g_convs+=1
                    if 'convblock' and 'shortcut' in key:
                        # if not 'deconv' in key:
                            g_conv_blocks+=1

                    # if 'deconvblock' and 'shortcut' in key:
                    #     g_deconv_blocks+=1
                        
                    if 'deconv' in key:
                        if not 'shortcut' in key:
                             g_deconvs+=1
                
                assert g_steps == g_convs +2*(g_deconv_blocks+g_conv_blocks)+ g_deconvs,'\nCheck keys in g_sizes, \n sum of generator steps do not coincide with sum of convolutional layers and convolutional blocks'



In [ ]:
g_convs

In [ ]:
g_deconv_blocks

In [ ]:
g_deconvs

In [ ]:
g_conv_blocks

In [ ]:
g_steps

In [ ]:
g_sizes_A['deconvblock_layer_0']